# **LAB 01: TRỰC QUAN HÓA DỮ LIỆU VỚI PYTHON**
#### Môn học: Trực quan hóa dữ liệu
#### Nhóm: 5


#### **A. Thu thập dữ liệu**

Trình bày các yêu cầu trong file pdf hướng dẫn

#### **B. Khám phá dữ liệu**

Trình bày các yêu cầu trong file pdf hướng dẫn

#### **C. Khám phá mối quan hệ trong dữ liệu**

#### **D. Xây dựng mô hình học máy**